In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2023/train_data.csv
/kaggle/input/widsdatathon2023/test_data.csv
/kaggle/input/widsdatathon2023/sample_solution.csv


In [3]:
train_raw = pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv', parse_dates=["startdate"])
test_raw = pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv', parse_dates=["startdate"])
submit = pd.read_csv('/kaggle/input/widsdatathon2023/sample_solution.csv')
target = 'contest-tmp2m-14d__tmp2m'

In [5]:
def rmse(actual, predicted):
    return mean_squared_error(actual, predicted, squared=False)

def location_nom(train, test):
    # Ref: https://www.kaggle.com/code/flaviafelicioni/wids-2023-different-locations-train-test-solved
    scale = 14

    train.loc[:,'lat']=round(train.lat,scale)
    train.loc[:,'lon']=round(train.lon,scale)
    test.loc[:,'lat']=round(test.lat,scale)
    test.loc[:,'lon']=round(test.lon,scale)

    all_df = pd.concat([train, test], axis=0)
    all_df['loc_group'] = all_df.groupby(['lat','lon']).ngroup()
    train = all_df.iloc[:len(train)]
    test = all_df.iloc[len(train):].drop(target, axis=1)
    
    return train, test

def categorical_encode(train, test):
    le = LabelEncoder()
    train['climateregions__climateregion'] = le.fit_transform(train['climateregions__climateregion'])
    test['climateregions__climateregion'] = le.transform(test['climateregions__climateregion'])
    return train, test
    
def fill_na(df):
    df = df.sort_values(by=['loc_group', 'startdate']).ffill()
    return df

def creat_new_featute(df):
    df['year'] = df['startdate'].dt.year
    df['month'] = df['startdate'].dt.month
    df['day_of_year'] = df['startdate'].dt.dayofyear
    # df['day_of_week'] = df['startdate'].dt.dayofweek
    # df['week_of_year'] = df['startdate'].dt.isocalendar().week
    return df

def feature_engineering(train_raw, test_raw):
    train, test = location_nom(train_raw, test_raw)
    train = fill_na(train)
    train = creat_new_featute(train)
    test = creat_new_featute(test)
    train, test = categorical_encode(train, test)

    drop_cols = ['index', 'startdate', 'lat', 'lon', target]
    features = [col for col in train.columns if col not in drop_cols]
    X = train[features]
    X_test = test[features]
    y = train[target]

    return X, y, X_test

In [6]:
X, y, X_test = feature_engineering(train_raw.copy(), test_raw.copy())

In [ ]:
# df_train = df_train.drop(['lat', 'lon','startdate', 'index'], axis=1)

In [ ]:
# inde = df_test['index']
# df_test = df_test.drop(['lat', 'lon', 'startdate', 'index'], axis=1)

In [ ]:
# le = preprocessing.LabelEncoder()
# le.fit(df_train['climateregions__climateregion'])
# df_train['climateregions__climateregion'] = le.transform(df_train['climateregions__climateregion'])

In [ ]:
# df_test['climateregions__climateregion'] = le.transform(df_test['climateregions__climateregion'])

In [ ]:
# y = df_train['contest-tmp2m-14d__tmp2m']

In [ ]:
y

In [ ]:
# X = df_train.copy()
# X = X.drop('contest-tmp2m-14d__tmp2m', axis=1)
# X = X.fillna(0)

In [ ]:
X.shape

In [ ]:
# x_test = df_test.copy()
# x_test = x_test.fillna(0)

In [7]:
input_shape = [X.shape[1]]
 
input_shape

[245]

In [8]:
model = tf.keras.Sequential([
 
    tf.keras.layers.Dense(units=128, activation='relu',
                          input_shape=input_shape),
    tf.keras.layers.Dense(units=64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(units=32, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(units=1)
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               31488     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 41,857
Trainable params: 41,857
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# model.compile(optimizer='adam', 
               
#               # MAE error is good for
#               # numerical predictions
#               loss='mae') 

In [9]:
model.compile(optimizer='adam', 
               
              loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError()]) 

In [20]:
losses = model.fit(X, y,
 
                   # it will use 'batch_size' number
                   # of examples per example
                   batch_size=256,
                   epochs=100,  # total epoch
 
                   )

Epoch 1/100
1468/1468 [==============================] - 6s 4ms/step - loss: 0.6507 - root_mean_squared_error: 0.8057
Epoch 2/100
1468/1468 [==============================] - 6s 4ms/step - loss: 0.6518 - root_mean_squared_error: 0.8063
Epoch 3/100
1468/1468 [==============================] - 6s 4ms/step - loss: 0.6533 - root_mean_squared_error: 0.8073
Epoch 4/100
1468/1468 [==============================] - 6s 4ms/step - loss: 0.6487 - root_mean_squared_error: 0.8044
Epoch 5/100
1468/1468 [==============================] - 6s 4ms/step - loss: 0.6517 - root_mean_squared_error: 0.8063
Epoch 6/100
1468/1468 [==============================] - 6s 4ms/step - loss: 0.6533 - root_mean_squared_error: 0.8073
Epoch 7/100
1468/1468 [==============================] - 6s 4ms/step - loss: 0.6465 - root_mean_squared_error: 0.8030
Epoch 8/100
1468/1468 [==============================] - 6s 4ms/step - loss: 0.6523 - root_mean_squared_error: 0.8066
Epoch 9/100
1468/1468 [==============================] -

In [21]:
y_pred = model.predict(X_test)

980/980 [==============================] - 1s 1ms/step


In [22]:
ypred = []
for i in y_pred:
    ypred.append(i[0])

In [23]:
ypred = np.array(ypred)

In [24]:
ypred

array([28.844387 , 28.887589 , 28.76523  , ...,  3.3547778,  3.661798 ,
        3.9244022], dtype=float32)

In [25]:
submit = pd.DataFrame()

In [27]:
submit['contest-tmp2m-14d__tmp2m'] = ypred
submit['index'] = test_raw['index']

In [28]:
submit 

,contest-tmp2m-14d__tmp2m,index
0,28.844387,375734
1,28.887589,375735
2,28.765230,375736
3,28.623676,375737
4,28.618172,375738
...,...,...
31349,3.513928,407083
31350,3.559047,407084
31351,3.354778,407085
31352,3.661798,407086


In [33]:
submit.to_csv('neural_network.csv', index = False)